In [1]:
pip install keras matplotlib numpy tensorflow kagglehub

In [2]:
# check GPU
!nvidia-smi

Sat Oct 11 07:02:38 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   38C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
# check tensorflow version
import tensorflow as tf
print(tf.__version__)

2.19.0


In [7]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import kagglehub

# Download latest version
path = kagglehub.dataset_download("paultimothymooney/chest-xray-pneumonia")

print("Path to dataset files:", path)

# Set paths to the dataset
training_dir = path + '/chest_xray/train'
testing_dir = path + '/chest_xray/test'

# Image data generator for pre-processing and augmenting images
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=30,
                                   zoom_range=0.2,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   brightness_range=[0.8, 1.2],  # Change brightness
                                   horizontal_flip=True,
                                   fill_mode="nearest"
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Create generators for training and testing
test_generator = test_datagen.flow_from_directory(
    testing_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)
train_generator = train_datagen.flow_from_directory(
    training_dir,
    target_size=(224, 224),  # Match input size of pre-trained models
    batch_size=32,
    class_mode='binary'  # Because it’s binary classification (Pneumonia vs Normal)
)


Using Colab cache for faster access to the 'chest-xray-pneumonia' dataset.
Path to dataset files: /kaggle/input/chest-xray-pneumonia
Found 624 images belonging to 2 classes.
Found 5216 images belonging to 2 classes.


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# Load MobileNetV2 pre-trained on ImageNet
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model so we don't train it
base_model.trainable = False

# Add custom classification layers on top
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(1, activation='sigmoid')  # Binary classification (Pneumonia or Normal)
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=30,  # Increase the number of epochs
    verbose=1
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
163/163 ━━━━━━━━━━━━━━━━━━━━ 173s 950ms/step - accuracy: 0.8086 - loss: 0.4095 - val_accuracy: 0.8237 - val_loss: 0.3485
Epoch 2/30
163/163 ━━━━━━━━━━━━━━━━━━━━ 118s 727ms/step - accuracy: 0.9115 - loss: 0.2140 - val_accuracy: 0.8686 - val_loss: 0.2786
Epoch 3/30
 38/163 ━━━━━━━━━━━━━━━━━━━━ 1:26 693ms/step - accuracy: 0.9332 - loss: 0.1776

In [ ]:
loss, accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:

# Unfreeze some layers of the base model
base_model.trainable = True

# Fine-tune starting from layer 100 (for example)
for layer in base_model.layers[:100]:
    layer.trainable = False

# Re-compile the model with a lower learning rate for fine-tuning
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Fine-tune the model
history_fine = model.fit(train_generator,
                         validation_data=test_generator,
                         epochs=20)  # Additional epochs for fine-tuning


In [ ]:
# prompt: save this trained model

model.save('/content/drive/MyDrive/pneumonia_classification_model.h5')

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Get predictions for the test set
test_generator.reset()
predictions = model.predict(test_generator)
predicted_classes = np.where(predictions > 0.5, 1, 0)  # Since it's a binary classification

# Get true labels
true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

# Classification report
report = classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report)

# Confusion matrix
cm = confusion_matrix(true_classes, predicted_classes)
print(cm)